In [1]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
import numpy as np


In [4]:
import pandas as pd

data = pd.read_csv("wsj.csv", encoding="ISO-8859-1")
data.head(4)

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,positivity,positivity:confidence,relevance,relevance:confidence,articleid,date,headline,positivity_gold,relevance_gold,text
0,842613455,False,finalized,3,12/5/2015 17:48,3.0,0.64,yes,0.640,wsj_398217788,8/14/1991,Yields on CDs Fell in the Latest Week,NaN,NaN,NEW YORK -- Yields on most certificates of dep...
1,842613456,False,finalized,3,12/5/2015 16:54,NaN,NaN,no,1.000,wsj_399019502,8/21/2007,The Morning Brief: White House Seeks to Limit ...,NaN,NaN,The Wall Street Journal Online</br></br>The Mo...
2,842613457,False,finalized,3,12/5/2015 1:59,NaN,NaN,no,1.000,wsj_398284048,11/14/1991,Banking Bill Negotiators Set Compromise --- Pl...,NaN,NaN,WASHINGTON -- In an effort to achieve banking ...
3,842613458,False,finalized,3,12/5/2015 2:19,NaN,0.00,no,0.675,wsj_397959018,6/16/1986,Manager's Journal: Sniffing Out Drug Abusers I...,NaN,NaN,The statistics on the enormous costs of employ...


In [5]:
data["text1"] = data["headline"] + " " +data["text"]

In [6]:
data.head(3)

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,positivity,positivity:confidence,relevance,relevance:confidence,articleid,date,headline,positivity_gold,relevance_gold,text,text1
0,842613455,False,finalized,3,12/5/2015 17:48,3.0,0.64,yes,0.64,wsj_398217788,8/14/1991,Yields on CDs Fell in the Latest Week,NaN,NaN,NEW YORK -- Yields on most certificates of dep...,Yields on CDs Fell in the Latest Week NEW YORK...
1,842613456,False,finalized,3,12/5/2015 16:54,NaN,NaN,no,1.00,wsj_399019502,8/21/2007,The Morning Brief: White House Seeks to Limit ...,NaN,NaN,The Wall Street Journal Online</br></br>The Mo...,The Morning Brief: White House Seeks to Limit ...
2,842613457,False,finalized,3,12/5/2015 1:59,NaN,NaN,no,1.00,wsj_398284048,11/14/1991,Banking Bill Negotiators Set Compromise --- Pl...,NaN,NaN,WASHINGTON -- In an effort to achieve banking ...,Banking Bill Negotiators Set Compromise --- Pl...


In [7]:
import pandas as pd
from bs4 import BeautifulSoup
from html import unescape

def clean_html(html):
    soup = BeautifulSoup(html, "html.parser")  # Parse the HTML
    text = soup.get_text()  # Extract text
    return unescape(text)  # Unescape HTML entities

# Apply the function to your DataFrame column
df['text'] = df['text'].apply(clean_html)

print(df[['text']])

                                                   text
0     NEW YORK -- Yields on most certificates of dep...
1     The Wall Street Journal OnlineThe Morning Brie...
2     WASHINGTON -- In an effort to achieve banking ...
3     The statistics on the enormous costs of employ...
4     NEW YORK -- Indecision marked the dollar's ton...
...                                                 ...
7995  Secretary of Commerce Charles W. Sawyer said y...
7996  U.S. stocks inched up last week, overcoming co...
7997  Ben S. Bernanke cleared a key hurdle Thursday ...
7998  The White House's push to contract out many fe...
7999  NEW YORK. April 17-Automobile stocks put on th...

[8000 rows x 1 columns]


In [9]:
data['text'] = data['text'].apply(lambda x: x.lower())
data

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,positivity,positivity:confidence,relevance,relevance:confidence,articleid,date,headline,positivity_gold,relevance_gold,text,text1
0,842613455,False,finalized,3,12/5/2015 17:48,3.0,0.6400,yes,0.6400,wsj_398217788,8/14/1991,Yields on CDs Fell in the Latest Week,NaN,NaN,new york -- yields on most certificates of dep...,Yields on CDs Fell in the Latest Week NEW YORK...
1,842613456,False,finalized,3,12/5/2015 16:54,NaN,NaN,no,1.0000,wsj_399019502,8/21/2007,The Morning Brief: White House Seeks to Limit ...,NaN,NaN,the wall street journal onlinethe morning brie...,The Morning Brief: White House Seeks to Limit ...
2,842613457,False,finalized,3,12/5/2015 1:59,NaN,NaN,no,1.0000,wsj_398284048,11/14/1991,Banking Bill Negotiators Set Compromise --- Pl...,NaN,NaN,washington -- in an effort to achieve banking ...,Banking Bill Negotiators Set Compromise --- Pl...
3,842613458,False,finalized,3,12/5/2015 2:19,NaN,0.0000,no,0.6750,wsj_397959018,6/16/1986,Manager's Journal: Sniffing Out Drug Abusers I...,NaN,NaN,the statistics on the enormous costs of employ...,Manager's Journal: Sniffing Out Drug Abusers I...
4,842613459,False,finalized,3,12/5/2015 17:48,3.0,0.3257,yes,0.6400,wsj_398838054,10/4/2002,Currency Trading: Dollar Remains in Tight Rang...,NaN,NaN,new york -- indecision marked the dollar's ton...,Currency Trading: Dollar Remains in Tight Rang...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,830985632,False,finalized,3,11/17/2015 18:16,7.0,0.3511,yes,0.6641,wapo_152454634,2/11/1952,"Sawyer Sees Strong Economy For 2 Years, Truce ...",NaN,NaN,secretary of commerce charles w. sawyer said y...,"Sawyer Sees Strong Economy For 2 Years, Truce ..."
7996,830985633,False,finalized,3,11/18/2015 8:10,NaN,NaN,no,1.0000,wapo_410349039,12/13/2009,Oil's losses are airlines' gains,NaN,NaN,"u.s. stocks inched up last week, overcoming co...",Oil's losses are airlines' gains U.S. stocks i...
7997,830985634,False,finalized,3,11/18/2015 12:12,NaN,NaN,no,1.0000,wapo_410346237,12/18/2009,Full Senate to vote on Bernanke; PANEL ADVANCE...,NaN,NaN,ben s. bernanke cleared a key hurdle thursday ...,Full Senate to vote on Bernanke; PANEL ADVANCE...
7998,830985635,False,finalized,3,11/18/2015 7:17,NaN,0.0000,no,0.6855,wapo_307808606,1/10/1995,Reinventing Opportunities,NaN,NaN,the white house's push to contract out many fe...,Reinventing Opportunities The White House's pu...


In [8]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd

# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform the data
labels = label_encoder.fit_transform(df['relevance'])

In [9]:
texts = df.text2.tolist()  # Your texts

# Tokenization
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

# Encoding texts
encodings = tokenizer(texts, truncation=True, padding=True, max_length=128, return_tensors="pt")

# Split data into train and validation sets
train_encodings, val_encodings, train_labels, val_labels = train_test_split(encodings['input_ids'], labels, test_size=0.2)

# Convert labels to tensors
train_labels = torch.tensor(train_labels)
val_labels = torch.tensor(val_labels)

# Create dataset objects
train_dataset = TensorDataset(train_encodings, train_labels)
val_dataset = TensorDataset(val_encodings, val_labels)


In [10]:
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=len(set(labels)))


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
import torch
from torch.utils.data import DataLoader
from transformers import AdamW

# Assuming model, train_dataset, and val_dataset are predefined
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)

optim = torch.optim.AdamW(model.parameters(), lr=5e-5)  # Use PyTorch's AdamW

# Training loop
for epoch in range(3):  # Number of epochs
    model.train()  # Set model to training mode
    for batch in train_loader:
        optim.zero_grad()  # Clear previous gradients

        # Assuming batch is a list or tuple, unpack directly by position
        input_ids, attention_masks, labels = batch
        
        # Move data to the correct device
        input_ids, attention_masks, labels = input_ids.to(device), attention_masks.to(device), labels.to(device).long()

        # Adjust the model call to include attention_masks
        outputs = model(input_ids, attention_mask=attention_masks, labels=labels)
        
        loss = outputs.loss
        loss.backward()  # Compute gradients
        optim.step()  # Update parameters

    print(f"Epoch {epoch+1}, Loss: {loss.item()}")

# Validation loop
model.eval()  # Set model to evaluation mode
correct, total = 0, 0  # Initialize variables for validation accuracy calculation
with torch.no_grad():  # No gradient needed for evaluation
    for batch in val_loader:
        # Assuming batch is a list or tuple, unpack directly by position
        input_ids, attention_masks, labels = batch

        # Move data to the correct device
        input_ids, attention_masks, labels = input_ids.to(device), attention_masks.to(device), labels.to(device).long()
        
        # Adjust the model call to include attention_masks
        outputs = model(input_ids, attention_mask=attention_masks)
        
        predictions = torch.argmax(outputs.logits, dim=1)
        correct += (predictions == labels).sum().item()
        total += labels.size(0)

print(f"Validation Accuracy: {correct / total:.4f}")


ValueError: not enough values to unpack (expected 3, got 2)